In [1]:
import json
import os
import monai
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

 missing cuda symbols while dynamic loading
 cuFile initialization failed


In [2]:
df = pd.read_csv("/home/jaalzate/Prostate_Cancer_TFM/Files/Own_data/Associated_Images_Unified.csv")

In [3]:
df

,subject,session,image_t2,image_dwi,image_adc,dep,ED,AF,TB,TR,PSA,VP,PIR,csPC,F_nacimiento,F_RM,partition,Derivative,VP_segmentation
0,sub-002299,ses-002477,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,82.0,NaN,2.0,NaN,32.490,NaN,NaN,1.0,1935-05-07,2013-05-25,train,False,NaN
1,sub-002299,ses-002477,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,82.0,NaN,2.0,NaN,32.490,NaN,NaN,1.0,1935-05-07,2013-05-25,train,False,NaN
2,sub-002299,ses-002477,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,82.0,NaN,2.0,NaN,32.490,NaN,NaN,1.0,1935-05-07,2013-05-25,train,False,NaN
3,sub-002299,ses-004352,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,82.0,NaN,2.0,NaN,0.014,38.884040,NaN,1.0,1935-05-07,2014-12-25,train,False,NaN
4,sub-002299,ses-004352,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,82.0,NaN,2.0,NaN,0.014,38.884040,NaN,1.0,1935-05-07,2014-12-25,train,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2881,sub-000128,ses-000128,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,7.0,70.0,NaN,1.0,NaN,9.380,72.000000,3.0,0.0,1946-05-06,2018-03-02,val,False,73.724678
2882,sub-000128,ses-000128,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,7.0,70.0,NaN,1.0,NaN,9.380,72.000000,3.0,0.0,1946-05-06,2018-03-02,val,False,73.724678
2883,sub-000173,ses-000173,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,72.0,NaN,1.0,NaN,5.370,49.879346,NaN,1.0,1940-08-02,2013-02-07,val,False,42.016346
2884,sub-000173,ses-000173,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,72.0,NaN,1.0,NaN,5.370,49.879346,NaN,1.0,1940-08-02,2013-02-07,val,False,42.016346


In [4]:
def process_row(row):
    path_t2 = row["image_t2"]
    path_dwi = row["image_dwi"]
    path_adc = row["image_adc"]

    shape_t2 = monai.transforms.LoadImage(image_only=True)(path_t2).shape
    shape_adc = monai.transforms.LoadImage(image_only=True)(path_adc).shape
    shape_dwi = monai.transforms.LoadImage(image_only=True)(path_dwi).shape
    

    return pd.Series(
        {
            "shape_t2": shape_t2,
            "shape_adc": shape_adc,
            "shape_dwi": shape_dwi,
        }
    )


# Aplica la función a cada fila del DataFrame utilizando apply
df[["shape_t2","shape_adc","shape_dwi"]] = df.progress_apply(
    process_row, axis=1
)

100%|██████████| 2886/2886 [16:28<00:00,  2.92it/s]


In [5]:
df

,subject,session,image_t2,image_dwi,image_adc,dep,ED,AF,TB,TR,...,PIR,csPC,F_nacimiento,F_RM,partition,Derivative,VP_segmentation,shape_t2,shape_adc,shape_dwi
0,sub-002299,ses-002477,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,82.0,NaN,2.0,NaN,...,NaN,1.0,1935-05-07,2013-05-25,train,False,NaN,"(512, 512, 26)","(256, 256, 20)","(256, 256, 35, 2)"
1,sub-002299,ses-002477,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,82.0,NaN,2.0,NaN,...,NaN,1.0,1935-05-07,2013-05-25,train,False,NaN,"(512, 512, 53)","(256, 256, 20)","(256, 256, 35, 2)"
2,sub-002299,ses-002477,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,82.0,NaN,2.0,NaN,...,NaN,1.0,1935-05-07,2013-05-25,train,False,NaN,"(512, 512, 26)","(256, 256, 20)","(256, 256, 35, 2)"
3,sub-002299,ses-004352,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,82.0,NaN,2.0,NaN,...,NaN,1.0,1935-05-07,2014-12-25,train,False,NaN,"(512, 512, 26)","(256, 256, 20)","(256, 256, 38, 2)"
4,sub-002299,ses-004352,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,82.0,NaN,2.0,NaN,...,NaN,1.0,1935-05-07,2014-12-25,train,False,NaN,"(512, 512, 26)","(256, 256, 20)","(256, 256, 38, 2)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2881,sub-000128,ses-000128,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,7.0,70.0,NaN,1.0,NaN,...,3.0,0.0,1946-05-06,2018-03-02,val,False,73.724678,"(384, 384, 27)","(150, 76, 27)","(268, 216, 50)"
2882,sub-000128,ses-000128,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,7.0,70.0,NaN,1.0,NaN,...,3.0,0.0,1946-05-06,2018-03-02,val,False,73.724678,"(256, 256, 56)","(150, 76, 27)","(268, 216, 50)"
2883,sub-000173,ses-000173,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,72.0,NaN,1.0,NaN,...,NaN,1.0,1940-08-02,2013-02-07,val,False,42.016346,"(512, 512, 28)","(256, 256, 21)","(256, 256, 31, 2)"
2884,sub-000173,ses-000173,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,72.0,NaN,1.0,NaN,...,NaN,1.0,1940-08-02,2013-02-07,val,False,42.016346,"(512, 512, 28)","(256, 256, 21)","(256, 256, 31, 2)"


In [16]:
#Check if any shape has 4 dimmensions
df[df["shape_t2"].apply(lambda x: len(x) == 4)]

,subject,session,image_t2,image_dwi,image_adc,dep,ED,AF,TB,TR,...,PIR,csPC,F_nacimiento,F_RM,partition,Derivative,VP_segmentation,shape_t2,shape_adc,shape_dwi
324,sub-002310,ses-002490,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,64.0,NaN,NaN,NaN,...,NaN,1.0,1946-02-28,2010-02-28,train,False,NaN,"(512, 512, 31, 2)","(256, 256, 23)","(256, 256, 35, 2)"
325,sub-002310,ses-002490,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,64.0,NaN,NaN,NaN,...,NaN,1.0,1946-02-28,2010-02-28,train,False,NaN,"(512, 512, 31, 2)","(256, 256, 23)","(256, 256, 35, 2)"
427,sub-001448,ses-001562,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,75.0,NaN,NaN,NaN,...,NaN,1.0,1938-05-02,2013-08-22,train,False,NaN,"(512, 512, 40, 2)","(256, 256, 27)","(256, 256, 27, 2)"
1151,sub-001334,ses-001440,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,73.0,NaN,3.0,NaN,...,NaN,1.0,1938-06-17,2014-05-13,train,False,NaN,"(512, 512, 29, 2)","(256, 256, 17)","(256, 256, 17, 2)"
1152,sub-001334,ses-001440,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,73.0,NaN,3.0,NaN,...,NaN,1.0,1938-06-17,2014-05-13,train,False,NaN,"(512, 512, 29, 2)","(256, 256, 17)","(256, 256, 17, 2)"
2461,sub-001124,ses-001218,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,69.0,NaN,2.0,NaN,...,NaN,1.0,1942-06-26,2005-08-10,train,False,NaN,"(512, 512, 26, 2)","(256, 256, 19)","(256, 256, 19, 3)"
2462,sub-001124,ses-001218,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,69.0,NaN,2.0,NaN,...,NaN,1.0,1942-06-26,2005-08-10,train,False,NaN,"(512, 512, 26, 2)","(256, 256, 19)","(256, 256, 19, 3)"
2538,sub-007342,ses-008075,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,75.0,NaN,4.0,NaN,...,5.0,0.0,1937-01-02,2009-01-05,train,False,NaN,"(512, 512, 29, 2)","(256, 256, 21)","(256, 256, 21, 3)"
2539,sub-007342,ses-008075,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,75.0,NaN,4.0,NaN,...,5.0,0.0,1937-01-02,2009-01-05,train,False,NaN,"(512, 512, 29, 2)","(256, 256, 21)","(256, 256, 21, 3)"
2574,sub-001316,ses-001420,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,/mnt/ceib/datalake/FISABIO_datalake/prueba/p00...,17.0,61.0,NaN,NaN,NaN,...,NaN,1.0,1944-10-02,2006-03-03,train,False,NaN,"(512, 512, 36, 2)","(256, 256, 18)","(256, 256, 18, 3)"


In [30]:
#Fix shapes

for i,row in df[df["shape_t2"].apply(lambda x: len(x) == 4)].iterrows():
    img = monai.transforms.LoadImage(image_only=True)(row["image_t2"])
    img_0 = img[...,0]
    parent_dir = os.path.dirname(row["image_t2"])

    #save image
    monai.transforms.SaveImage(output_dir=parent_dir, output_ext=".nii.gz",output_postfix="",channel_dim=None,separate_folder=False)(img_0)

2024-07-02 10:28:22,202 INFO image_writer.py:197 - writing: /mnt/ceib/datalake/FISABIO_datalake/prueba/p0052021_reborn/sub-002310/ses-002490/mim-mr/anat/sub-002310_ses-002490_acq-propeller_run-9_bp-prostate_vp-ax_T2w.nii.gz
2024-07-02 10:28:23,448 INFO image_writer.py:197 - writing: /mnt/ceib/datalake/FISABIO_datalake/prueba/p0052021_reborn/sub-002310/ses-002490/mim-mr/anat/sub-002310_ses-002490_acq-propeller_run-900_bp-prostate_vp-ax_T2w.nii.gz
2024-07-02 10:28:24,823 INFO image_writer.py:197 - writing: /mnt/ceib/datalake/FISABIO_datalake/prueba/p0052021_reborn/sub-001448/ses-001562/mim-mr/anat/sub-001448_ses-001562_acq-propeller_run-4_bp-prostate_vp-ax_T2w.nii.gz
2024-07-02 10:28:26,123 INFO image_writer.py:197 - writing: /mnt/ceib/datalake/FISABIO_datalake/prueba/p0052021_reborn/sub-001334/ses-001440/mim-mr/anat/sub-001334_ses-001440_acq-propeller_run-700_bp-prostate_vp-ax_T2w.nii.gz
2024-07-02 10:28:27,241 INFO image_writer.py:197 - writing: /mnt/ceib/datalake/FISABIO_datalake/prue